**Desarrollo de funciones para realizar las consultas.**

El cliente solicita que desde la API se puedan realizar seis consultas diferentes a nuestros datos. Las consultas son:
1. Película con mayor duración según el año, la plataforma de streaming y su tipo de duración (min).
2. Cantidad de películas por plataforma con un puntaje mayor a XX en un año determinado.
3. Cantidad de películas que se pueden encontrar según la plataforma.
4. Actor que más se repite en los elencos según la plataforma y el año de la producción cinematográfica.
5. La cantidad de contenidos/productos que se publicó por país y año.
6. La cantidad total de contenidos según el rating de audiencia dado (para que publico fue clasificada la pelicula).


En este apartado, se encontrarán las funciones que se realizaron para poder navegar dentro del DataFrame y obtener las respuestas a consultas que se mencionaron anteriormente.

In [1]:
# Antes de comenzar, importamos la librería a utilizar, llamada Pandas.
import pandas as pd

In [2]:
# Primero importamos el archivo para que podamos realizar las consultas necesarias.
plataformas_df = pd.read_csv(r'.\Dataset\Streaming_services_clean.csv')

Se valida que se hayan cargado los datos correctamente.

In [3]:
plataformas_df.head()

,id,type,title,director,cast,country,date_added,release_year,rating,duration_int,duration_type,listed_in,description,userId,score,timestamp,mean_score
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,543,5.0,2003-07-30 20:23:02,3.467131
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110,min,"drama, international",a metro family decides to fight a cyber crimin...,321,3.0,2007-06-10 04:59:41,3.548682
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74,min,"action, drama, suspense",after a man discovers his wife is cheating on ...,582,5.0,2011-09-24 05:49:30,3.500000
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69,min,documentary,"pink breaks the mold once again, bringing her ...",151,1.5,2004-04-08 19:54:41,3.538055
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45,min,"drama, fantasy",teenage matt banting wants to work with a famo...,24,5.0,2001-01-20 06:34:51,3.478992


----------

Consulta N°1: Película con mayor duración según el año, la plataforma de streaming y su tipo de duración (min).

In [12]:
def get_max_duration(anio, plataforma, dtype):
    
    # Se realiza una copia de la base de datos para que, en el caso de que pase, no sufra modificaciones la base de datos original.
    gmd_p = plataformas_df 
    
    # Se crean los filtros de  dependiendo la plataforma, el año y si es tipo 'min' o 'seasons'.
    if plataforma.lower() == 'amazon prime':
        if dtype == 'min':
            gmd_p = gmd_p[(gmd_p['type'] == 'movie') & (gmd_p['release_year'] == anio) & (gmd_p['duration_type'] == dtype) & (gmd_p['id'].apply(lambda x: 'a' in x))]
        elif dtype == 'season':
            return 'La función sólo permite buscar películas con máxima duración. Para realizar la búsqueda ingrese: min.'
    
    elif plataforma.lower() == 'disney plus':
        if dtype == 'min':
            gmd_p = gmd_p[(gmd_p['type'] == 'movie') & (gmd_p['release_year'] == anio) & (gmd_p['duration_type'] == dtype) & (gmd_p['id'].apply(lambda x: 'd' in x))]
        elif dtype == 'season':
            return 'La función sólo permite buscar películas con máxima duración. Para realizar la búsqueda ingrese: min.'
    
    elif plataforma.lower() == 'hulu':
        if dtype == 'min':
            gmd_p = gmd_p[(gmd_p['type'] == 'movie') & (gmd_p['release_year'] == anio) & (gmd_p['duration_type'] == dtype) & (gmd_p['id'].apply(lambda x: 'h' in x))]
        elif dtype == 'season':
            return 'La función sólo permite buscar películas con máxima duración. Para realizar la búsqueda ingrese: min.'
    
    elif plataforma.lower() == 'netflix':
        if dtype == 'min':
            gmd_p = gmd_p[(gmd_p['type'] == 'movie') & (gmd_p['release_year'] == anio) & (gmd_p['duration_type'] == dtype) & (gmd_p['id'].apply(lambda x: 'n' in x))]
        elif dtype == 'season':
            return 'La función sólo permite buscar películas con máxima duración. Para realizar la búsqueda ingrese: min.'
    
    # En caso de que se consulte otra plataforma que no esté cargada en la base de datos, se retornará este mensaje explicativo.
    else: 
        print('No se encuentra en nuestro dataset la plataforma que desea buscar. Cabe aclarar que sólo tenemos registros pertenecientes a: Amazon Prime, Disney Plus, Hulu y Netflix. Disculpe las molestias ocasiondas.')
        return None
    
    # Se crea el filtro para que se obtenga la fila de la película o serie que tiene más duración.
    gmd_p_copia = gmd_p[gmd_p['duration_int'] == (gmd_p['duration_int'].max())]
    
    # Por último, imprime un mensaje donde contiene los resultados de la consulta.
    return {'pelicula': gmd_p_copia.iloc[0,2]}

Se verifica que la función devuelva la consulta correctamente. A modo de ejemplo, se utilizarán los valores: 2018, 'Hulu', 'min'.

In [13]:

get_max_duration(2018,'Hulu','min')

{'pelicula': 'the house that jack built'}

Consulta N°2: Cantidad de películas por plataforma con un puntaje mayor a XX en un año determinado.

In [56]:
def get_score_count(plataforma, scored, anio):
    
    # Se realiza una copia de la base de datos para que, en el caso de que pase, no sufra modificaciones la base de datos original.
    gsc_p= plataformas_df
    
    # Se crea el filtro para que se obtenga el resultado correcto segun el año de lanzamiento de la película/serie y que tenga una puntuación mayor a la que establezca el usuario.
    gsc_p = gsc_p[(gsc_p['mean_score'] >= scored) & (gsc_p['release_year'] == anio)]
    
    # Se crea el filtro para que pueda discriminar según la plataforma a buscar.
    if plataforma.lower() == 'amazon prime':
        gsc_pc = gsc_p[gsc_p['id'].apply(lambda x: 'a' in x)]

    elif plataforma.lower() == 'disney plus':
        gsc_pc = gsc_p[gsc_p['id'].apply(lambda x: 'd' in x)]
    
    elif plataforma.lower() == 'hulu':
        gsc_pc = gsc_p[gsc_p['id'].apply(lambda x: 'h' in x)]

    elif plataforma.lower() == 'netflix':
        gsc_pc = gsc_p[gsc_p['id'].apply(lambda x: 'n' in x)]

    # En caso de que se consulte otra plataforma que no esté cargada en la base de datos, se retornará este mensaje explicativo.
    else:
        print('No se encuentra en nuestro dataset la plataforma que desea buscar. Cabe aclarar que sólo tenemos registros pertenecientes a: Amazon Prime, Disney Plus, Hulu y Netflix. Disculpe las molestias ocasiondas')
    
    # Por último, imprime un mensaje donde contiene los resultados de la consulta.
    return {
        'plataforma': plataforma,
        'cantidad': gsc_pc.shape[0],
        'anio': anio,
        'score': scored
    }

Se verifica que la función devuelva la consulta correctamente. A modo de ejemplo, se utilizarán los valores: 'Netflix', 3 , 2016.

In [16]:
get_score_count('netflix',3.5,2016)

{'plataforma': 'netflix', 'cantidad': 677, 'anio': 2016, 'score': 3.5}

Consulta N°3: Cantidad de películas que se pueden encontrar según la plataforma.

In [54]:
def get_count_platform(plataforma):
    
    # Se realiza una copia de la base de datos para que, en el caso de que pase, no sufra modificaciones la base de datos original.
    gcp_p = plataformas_df
    
    # Se crea el filtro para que pueda discriminar según la plataforma a buscar.
    if plataforma.lower() == 'amazon prime':
        gcp_pc= gcp_p[gcp_p['id'].apply(lambda x: 'a' in x)]

    elif plataforma.lower() == 'disney plus':
        gcp_pc = gcp_p[gcp_p['id'].apply(lambda x: 'd' in x)]
    
    elif plataforma.lower() == 'hulu':
        gcp_pc = gcp_p[gcp_p['id'].apply(lambda x: 'h' in x)]

    elif plataforma.lower() == 'netflix':
        gcp_pc = gcp_p[gcp_p['id'].apply(lambda x: 'n' in x)]
    
    # En caso de que se consulte otra plataforma que no esté cargada en la base de datos, se retornará este mensaje explicativo.
    else:
        print('No se encuentra en nuestro dataset la plataforma que desea buscar. Cabe aclarar que sólo tenemos registros pertenecientes a: Amazon Prime, Disney Plus, Hulu y Netflix. Disculpe las molestias ocasiondas')
    
    # Por último, imprime un mensaje donde contiene los resultados de la consulta.
    return {'plataforma': plataforma, 'peliculas': gcp_pc.shape[0]}

Se verifica que funcione de manera correcta. A modo de ejemplo, se utilizarán los valores: 'Disney Plus'.

In [55]:
get_count_platform('Disney Plus')

{'plataforma': 'Disney Plus', 'peliculas': 1450}

Consulta N°4: Actor que más se repite en los elencos según la plataforma y el año de la producción cinematográfica.

In [3]:
def get_actor(plataforma, anio):
    
    # Se realiza una copia de la base de datos para que, en el caso de que pase, no sufra modificaciones la base de datos original.
    ga_p = plataformas_df
    
    # Se crea el filtro para que pueda discriminar según la plataforma, y luego obtener el valor que más se repite. 
    if plataforma.lower() == 'amazon prime':
        ga_pc= ga_p[(ga_p['release_year'] == anio) & (ga_p['id'].apply(lambda x: 'a' in x))]
        ga_pca= ga_pc['cast'].value_counts().idxmax()
        aux = ga_pc.loc[ga_pc['cast'] == ga_pca].count()[0]

    elif plataforma.lower() == 'disney plus':
        ga_pc = ga_p[(ga_p['release_year'] == anio) & (ga_p['id'].apply(lambda x: 'd' in x))]
        ga_pca= ga_pc['cast'].value_counts().idxmax()
        aux = ga_pc.loc[ga_pc['cast'] == ga_pca].count()[0]

    elif plataforma.lower() == 'hulu':
        ga_pc = ga_p[(ga_p['release_year'] == anio) & (ga_p['id'].apply(lambda x: 'h' in x))]
        ga_pca= ga_pc['cast'].value_counts().idxmax()
        aux = ga_pc.loc[ga_pc['cast'] == ga_pca].count()[0]

    elif plataforma.lower() == 'netflix':
        ga_pc = ga_p[(ga_p['release_year'] == anio) & (ga_p['id'].apply(lambda x: 'n' in x))]
        ga_pca= ga_pc['cast'].value_counts().idxmax()
        aux = ga_pc.loc[ga_pc['cast'] == ga_pca].count()[0]
        
    # En caso de que se consulte otra plataforma que no esté cargada en la base de datos, se retornará este mensaje explicativo.
    else:
        print('No se encuentra en nuestro dataset la plataforma que desea buscar. Cabe aclarar que sólo tenemos registros pertenecientes a: Amazon Prime, Disney Plus, Hulu y Netflix. Disculpe las molestias ocasiondas')
    
    # Por último, imprime un mensaje donde contiene los resultados de la consulta.
    return {
        'plataforma': plataforma,
        'anio': anio,
        'actor': ga_pca,
        'apariciones': aux
    }

Se verifica que la función devuelva la consulta correctamente. A modo de ejemplo, se utilizarán los valores: 'Amazon Prime', 2019.

In [4]:
get_actor('Amazon Prime', 2017)

{'plataforma': 'Amazon Prime',
 'anio': 2017,
 'actor': 'maggie binkley',
 'apariciones': 16}

Consulta N°5: La cantidad de contenidos/productos que se publicó por país y año.

In [5]:
def prod_per_county(tipo,pais,anio):
    
    # Se realiza una copia de la base de datos para que, en el caso de que pase, no sufra modificaciones la base de datos original.
    ppc_p = plataformas_df

    # Se crea un condicional en caso de que se ingrese un valor erróneo en el parámetro 'tipo'.
    if tipo not in ('movie','tv show'):
        
        return 'La función sólo acepta el tipo como movie o tv show. Ingrese algún formato válido.'
    
    else:
        
        # Se crea el filtro para que pueda discriminar según el tipo ('movie' o 'tv show'), pais y anio de la producción cinematográfica.
        ppc_pa = ppc_p[(ppc_p['type'] == tipo) & (ppc_p['country'] == pais) & (ppc_p['release_year'] == anio)]

        # Por último, imprime un mensaje donde contiene los resultados de la consulta.
        return {'pais': pais, 'anio': anio, 'peliculas': ppc_pa.count()[0]}

Se verifica devuelva la consulta eficientemente. A modo de ejemplo, se utilizarán los valores: 'tv show','united states', 2017.

In [6]:
prod_per_county('tv show','united states',2017)

{'pais': 'united states', 'anio': 2017, 'peliculas': 125}

Consulta N°6: La cantidad total de contenidos según el rating de audiencia dado (para que publico fue clasificada la pelicula).

In [70]:
def get_contents(rating):

    # Se realiza una copia de la base de datos para que, en el caso de que pase, no sufra modificaciones la base de datos original.
    gc_p = plataformas_df

    # Se crea el filtro para que pueda discriminar por el rating que eligió el usuario.
    gc_pr = gc_p[(gc_p['rating'] == rating)]

    # Por último, imprime un mensaje donde contiene los resultados de la consulta.
    return {'rating': rating, 'contenido': gc_pr.count()[0]}

Se valida que la función se ejecute de forma correcta. Se utilizara el parámetro '13+' a modo de ejemplo.

In [69]:
get_contents('13+')

{'rating': '13+', 'contenido': 2117}